# Loading Data

In [1]:
data <- read.csv("D:/Research/keyword/Keyword/All/12 journals.csv")
nrow(data)
# str(data)

[1] 5875

# Transforming Variables

In [2]:
na_index <- which(is.na(data$cited.by))
na_index
data[na_index, "cited.by"] <- 0

data$cited.by.positive <- 0

for (row in 1:nrow(data))
    {if (data[row,"cited.by"]   > 0 ) data[row,"cited.by.positive"] <- 1 }
     
data$cited.by.positive <- as.factor(data$cited.by.positive)
#data$cited.by.positive
summary(data$cited.by.positive)

integer(0)

0    1 
 859 5016

# Data Cleaning

In [7]:
# nrow(data)
# summary(data$cited.by_scaled)

# data <- data[data$cited.by_scaled <4,]
# nrow(data)
# summary(data$cited.by_scaled)

In [20]:
# cited.by.table <- table(data$cited.by)
# # cited.by.table
# # round(prop.table(cited.by.table),2) # row percentages 
# rbind(cited.by.table,round(prop.table(cited.by.table),2))

# ##------------------------------------
# cited.by.table <- table(log(data$cited.by_zero))
# # cited.by.table
# # round(prop.table(cited.by.table),2) # row percentages 
# rbind(cited.by.table,round(prop.table(cited.by.table),2))

# Creating New Target Variables

In [8]:
# data$cited.by_zero <- 0


# for (row in 1:nrow(data))
#     {if (data[row,"cited.by"] == 0 ) data[row,"cited.by_zero"] <- data[row,"cited.by"] + 0.5
#     else data[row,"cited.by_zero"] <- data[row,"cited.by"] }
     
# #hist(log(data$cited.by_zero))
    
     
# hist   (log(data$cited.by_zero))
# #barplot(log(data$cited.by_zero))


# Part 2: Predicting no citation(0) and positive citation(1)

In [11]:
sum(data$page.count<0)
sum(data$no_references<0)
sum(data$no_authors<0)

[1] 0

[1] 0

[1] 0

In [7]:
#names(data)
pred_numeric <- c("abstract_length" ,"page.count","title_length", "abstract_ratio" ,"no_references","no_authors"
,"numkeys","article_age","weight_ratio")
pred_cat     <- c("quartile","cited.by.positive")

pred_data  <- data[,c(pred_numeric,pred_cat)]

logistic   <- glm(cited.by.positive ~ ., abstract_length +log(page.count)+title_length+ abstract_ratio 
                  +log(no_references)+log(no_authors)+numkeys+article_age+weight_ratio
                  ,
                  family=binomial(link='logit'), data=pred_data)
summary(logistic)
#ound(logistic$coeff,3)
#summary(logistic)$coefficients[,4] 

Warning message:
In eval(expr, envir, enclos): non-integer #successes in a binomial glm!Warning message:
: glm.fit: algorithm did not convergeWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurred


Call:
glm(formula = cited.by.positive ~ ., family = binomial(link = "logit"), 
    data = pred_data, weights = abstract_length + log(page.count) + 
        title_length + abstract_ratio + log(no_references) + 
        log(no_authors) + numkeys + article_age + weight_ratio)

Deviance Residuals: 
   Min      1Q  Median      3Q     Max  
-188.1     0.0     0.0     0.0     0.0  

Coefficients:
                  Estimate Std. Error    z value Pr(>|z|)    
(Intercept)      2.979e+14  4.406e+05  6.762e+08   <2e-16 ***
abstract_length -3.095e+11  1.111e+03 -2.784e+08   <2e-16 ***
page.count       9.570e+12  1.752e+04  5.463e+08   <2e-16 ***
title_length    -7.790e+12  1.549e+04 -5.030e+08   <2e-16 ***
abstract_ratio   1.775e+15  2.684e+06  6.614e+08   <2e-16 ***
no_references    3.738e+12  2.986e+03  1.252e+09   <2e-16 ***
no_authors       1.261e+13  4.521e+04  2.789e+08   <2e-16 ***
numkeys         -1.035e+13  5.100e+04 -2.030e+08   <2e-16 ***
article_age      2.025e+14  2.193e+04  9.237e+09

In [10]:
class(data$cited.by.positive)
class(data$quarter)
table(data$cited.by.positive, data$quarter)

[1] "factor"

[1] "factor"

   
      q1   q2   q3   q4
  0  251  298  250   60
  1 3727  865  349   75

# Model Selection

In [5]:
# logistic_step <- step(logistic)
# logistic_step

In [6]:
# logistic_final <- glm(formula = cited.by.positive ~ abstract_length + page.count + 
#     abstract_ratio + no_references + no_authors + article_age + 
#     source.title, family = binomial(link = "logit"), data = pred_data)

# summary(logistic_final)

In [8]:
require(car,quietly=TRUE)
qqPlot(logistic, main="QQ Plot") #qq plot for studentized resid 
# qqPlot(linear_reg0, main="QQ Plot") #qq plot for studentized resid 


ERROR: Error in qqPlot.glm(logistic, main = "QQ Plot"): QQ plot for studentized residuals not available for glm


# Outlier Detection

In [13]:
# influenceIndexPlot <- influenceIndexPlot(linear_reg, id.n=10)
# str(influenceIndexPlot)
# outlierTest(linear_reg)
# data[c(903,908,925),]

In [14]:
# # Influence Plot 
# influencePlot(linear_reg,	id.method="identify", main="Influence Plot", 
#               sub="Circle size is proportial to Cook's Distance",id.n=10 )

# Box-Cox Transformation

In [16]:
# require(MASS, quietly=TRUE)
# require(car, quietly=TRUE)

# boxCox <- boxCox(logistic, lambda = seq(-0.1, 0.1, by=0.1))
# summary(powerTransform(logistic))


# Multi-Collinearity

In [17]:
require(car,quietly=TRUE)
# Evaluate Collinearity

vif(logistic) # variance inflation factors 
# ??vif
sqrt(vif(logistic)) > 2 # problem?
##---------------------------------------------


article_age      page.count abstract_length    title_length  abstract_ratio 
       1.680864        1.315781        1.096587        1.061121        1.064322 
  no_references  no_authors_cat    source.title 
       1.624516        1.044373        1.248188

article_age      page.count abstract_length    title_length  abstract_ratio 
          FALSE           FALSE           FALSE           FALSE           FALSE 
  no_references  no_authors_cat    source.title 
          FALSE           FALSE           FALSE

In [23]:
round(confint(logistic),2)

Waiting for profiling to be done...


,2.5 %,97.5 %
(Intercept),-1.28,-0.02
abstract_length,0,0
page.count,-0.08,-0.02
title_length,-0.03,0.01
abstract_ratio,1.66,8.83
no_references,0.01,0.02
no_authors,-0.01,0.12
weight_ratio,0.34,6.94
numkeys,-0.08,0.08
article_age,0.6,0.7


In [11]:
save.image()